# This notebook demos a diffusion stencil

Here follows the diffusion stencil

In [ ]:
from dusk.script import *


@stencil
def diffusion(
    TV: Field[Vertex],
    TE: Field[Edge], TEinit: Field[Edge], TE_t: Field[Edge], TEnabla2: Field[Edge],
    inv_primal_edge_length: Field[Edge], inv_vert_vert_length: Field[Edge], nnbhV: Field[Vertex],
    boundary_edge: Field[Edge],
    kappa: Field[Edge], dt: Field[Edge]
) -> None:

    with levels_upward:
       # initialize
        TEinit = TE
    
        # predict
        TE = TEinit + 0.5*dt*TE_t

        # interpolate temperature from edges to vertices
        TV = sum_over(Vertex > Edge, TE) / nnbhV

        # compute nabla2 using the finite differences
        TEnabla2 = sum_over(
            Edge > Cell > Vertex,
            TV,
            weights=[
                inv_primal_edge_length ** 2.,
                inv_primal_edge_length ** 2.,
                inv_vert_vert_length ** 2.,
                inv_vert_vert_length ** 2.,
            ],
        )
        TEnabla2 = TEnabla2 - (
            (2.0 * TE * inv_primal_edge_length ** 2.)
            + (2.0 * TE * inv_vert_vert_length ** 2.)
        )
    
        # build ODEs
        if (boundary_edge):
            TE_t = 0.
        else:
            TE_t = kappa*TEnabla2
    
        # correct
        TE = TEinit + dt*TE_t


Then we can use dusk's Python API to convert the stencils to SIR. This API can also invoke dawn to compile SIR to C++ which we will write to `diffusion_cxx-naive.cpp`:

In [ ]:
from dusk.transpile import callable_to_pyast, pyast_to_sir, sir_to_json
with open("diffusion.sir", "w+") as f:
    sir = pyast_to_sir(callable_to_pyast(diffusion))
    f.write(sir_to_json(sir))
!dawn-opt diffusion.sir | dawn-codegen -b naive-ico -o diffusion_cxx-naive.cpp

The generated C++ code also requires a driver which is already setup for this demo. With the driver code we can generate an executable `runner`:

In [ ]:
!make

First, we put the runner into test mode to ensure that the computed kernel is correct:

In [ ]:
!./runner test

If the tester reported that your dusk stencil works correctly, you can now run the complete stencil and visualize

In [ ]:
!./runner run

In [ ]:
%%capture
import prep_animation as pa
import matplotlib.animation as animation
from IPython.display import HTML
animate = animation.FuncAnimation(pa.fig, pa.update_plot, len(pa.out_files), fargs=(pa.zarray, pa.plot), interval = 50)

In [ ]:
HTML(animate.to_html5_video())